# The BATTLE OF NEIGHBORHOODS
## Selecting Neighborhood of Houston
## 1	Introduction/Business Problem

### 1.1	Background

Houston is the largest city in Texas, the fourth-largest in America, and brimming with opportunity, excitement, culture and history. A diverse city where more than 145 languages are spoken results in a food, art and community fusion unmatched in other destinations.
Unparalleled performing arts and nightlife activity keep this city running 24/7. The city’s relatively compact Downtown includes the Theater District, home to the renowned Houston Grand Opera, and the Historic District, with 19th-century architecture and upscale restaurants. Tucked between the bustling Midtown and Medical Center areas, rests Houston's leafy, park-like Museum District. Nineteen world-class institutions, including the Menil Collection and Museum of Fine Arts, Houston are clustered in this area, drawing seven million visitors to the district each year.	

### 1.2	Business Problem
		
As a visitor who has never been to Houston before, it is always difficult to find out best places to visit and find something which really suits their liking in the restricted time period. The visitor may enjoy Houston's outstanding performing and visual arts venues. Try one of the countless restaurants available, offering cuisine in everything from Tex Mex and South American to Middle Eastern and Vietnamese. For sports fans we have local teams representing all major sports. Do some shopping; Houston offers something to fit every budget - from the exclusive shops in Houston's Uptown area to the outlet malls just outside the City. 
Every person who visits Houston faces this problem to find suitable top-rated locations of their liking.  Therefore, as part of this Capstone project I tried solving this problem by segregate different neighborhoods to find different types of locations based on the surrounding.

### 1.3	Interest

We will use data science to generate and clustering the neighborhood in Houston. The advantages and feature of venue categories in each Houston neighborhood will then be clearly expressed so that the best neighborhood to visit can be chose by our target tourist.

## 2	Data

### 2.1	Data Sources

Following data sources are used for the analysis of neighborhoods of Houston:
1.	The name, location and boundaries for all the Houston neighborhoods can be retrieved from the following online open data : 
https://opendata.arcgis.com/datasets/deae15e7f3db4b198946c17f0d169c39_3.geojson
2.	Get the latitude and longitude values of Houston using geopy library
3.	Explore the venues and neighborhoods using Foursquare API


### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 8.3MB/s  eta 0:00:01
Libraries imported.


### Collect Data

In [2]:
!wget -q -O 'houston_data.json' https://opendata.arcgis.com/datasets/deae15e7f3db4b198946c17f0d169c39_3.geojson
print('Data downloaded!')

Data downloaded!


### Load and explore the data

In [3]:
with open('houston_data.json') as json_data:
    houston_data = json.load(json_data)

Let's take a quick look at the data.

In [4]:
houston_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 1,
    'PERIMETER': 16572.025,
    'POLYID': 60,
    'SNBNAME': 'FOURTH WARD',
    'GLOBALID': '{EDCA8617-F41A-4E57-8138-02BFCD50E562}',
    'cohgisCOHGISSNBRAREA': 0,
    'cohgisCOHGISSNBRLEN': 0,
    'COUNCIL_ACTIVE': None,
    'RECOGNITION_DATE': None,
    'SnbrInfoURL': 'http://www.houstontx.gov/superneighborhoods/60.html',
    'WeCan': 'Y',
    'Top10': None,
    'CEA_FLAG': 'N',
    'Shapearea': 12969824.766338602,
    'Shapelen': 16572.0260242156},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-95.3858120191703, 29.76157948522655],
      [-95.3857510351169, 29.759096134455724],
      [-95.38575701432947, 29.7589274318535],
      [-95.38581341906735, 29.75809220348262],
      [-95.38571553175525, 29.757628886192457],
      [-95.38567397795319, 29.753574178823797],
      [-95.38575841615513, 29.753438202654888],
      [-95.3857221276755, 29.750280400839365],
      [-95.38400247

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [5]:
neighborhoods_data = houston_data['features']

### 2.2	Data Cleaning
The data of Houston neighborhoods is imported and transformed to pandas dataframe.
The latitude and longitude of each Houston neighborhood can be calculated by taking the average center of all of its shape boundary points. If a neighborhood is defined by two or two more shape areas, only the first shape area will be used for the calculation of the latitude and longitude.


### Tranform the data into a pandas dataframe

Let's start by creating an empty dataframe.

In [6]:
# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Let's loop through the data and fill the dataframe one row at a time.

In [7]:
for data in neighborhoods_data:
    neighborhood_name = data['properties']['SNBNAME'] 
        
    pd_shape = pd.DataFrame(data['geometry']['coordinates'])
    
    # The provided data use shape to define neighborhood. let's use the average center point to represent the neighborhood location
    neighborhood_latlon=pd.DataFrame( pd.DataFrame(pd_shape.iloc[0])[0].tolist()).mean(axis=0)
    
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Let's check the number of neighborhoos in this analysis.

In [8]:
neighborhoods.shape

(88, 3)

Let's print the first 10 neighborhoods in this analysis.

In [9]:
neighborhoods.head(10)

,Neighborhood,Latitude,Longitude
0,FOURTH WARD,29.759108,-95.380204
1,SECOND WARD,29.751715,-95.331634
2,DOWNTOWN,29.758336,-95.353891
3,CLINTON PARK TRI-COMMUNITY,29.739786,-95.255946
4,GREATER UPTOWN,29.771256,-95.490082
5,GREATER INWOOD,29.881288,-95.487484
6,GREATER HOBBY AREA,29.625936,-95.291085
7,GOLFCREST / BELLFORT / REVEILLE,29.653929,-95.298948
8,ELDRIDGE / WEST OAKS,29.737417,-95.670287
9,WASHINGTON AVENUE COALITION / MEMORIAL PARK,29.765147,-95.418188


## 3 Methodology
In this project, we will devote our efforts on detecting and identifying the top common venue categories of each neighborhood in Houston.

### Use geopy library to get the latitude and longitude values of Houston

In [10]:
address = 'Houston, TX'

geolocator = Nominatim(user_agent="houston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Houston are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Houston are 29.7589382, -95.3676974.


### Create a map of Houston with neighborhoods superimposed on top.

In [11]:
# create map of New York using latitude and longitude values
map_houston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston)  
    
map_houston

### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = '5DHWS1YO2QADABIXLXTJPXV1MBQLNQLPISX4CBEMIP1KNKLR' # your Foursquare ID
CLIENT_SECRET = 'O4ODQCYFCNH2DW2WI52PE5NUN23LNSU5UUHB5Q4BLH3S1WMR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5DHWS1YO2QADABIXLXTJPXV1MBQLNQLPISX4CBEMIP1KNKLR
CLIENT_SECRET:O4ODQCYFCNH2DW2WI52PE5NUN23LNSU5UUHB5Q4BLH3S1WMR


### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [13]:
neighborhoods.loc[0, 'Neighborhood']

'FOURTH WARD'

Get the neighborhood's latitude and longitude values.

In [14]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of FOURTH WARD are 29.759108401057734, -95.38020398569252.


Now, let's get the top 100 venues that are in Fourth Ward within a radius of 2000 meters.

In [15]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5DHWS1YO2QADABIXLXTJPXV1MBQLNQLPISX4CBEMIP1KNKLR&client_secret=O4ODQCYFCNH2DW2WI52PE5NUN23LNSU5UUHB5Q4BLH3S1WMR&v=20180605&ll=29.759108401057734,-95.38020398569252&radius=2000&limit=100'

Send the GET request and examine the results

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9e5383aba297001bd5a71f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Fourth Ward',
  'headerFullLocation': 'Fourth Ward, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 225,
  'suggestedBounds': {'ne': {'lat': 29.77710841905775,
    'lng': -95.35950821600734},
   'sw': {'lat': 29.741108383057718, 'lng': -95.4008997553777}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bff00bcca1920a16f32ec81',
       'name': 'Eleanor Tinsley Park',
       'location': {'address': '500 Allen Parkway',
        'lat': 29.761439843452138,
        'lng': -95.37927095656195,
        'labeledL

Let's borrow the get_category_type function from the Foursquare lab.

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eleanor Tinsley Park,Park,29.761440,-95.379271
1,Buffalo Bayou Walk,Trail,29.762177,-95.375844
2,Buffalo Bayou Cistern,Public Art,29.762214,-95.375873
3,Oporto Fooding House & Wine,Portuguese Restaurant,29.753179,-95.380243
4,Cafe Poetes,Café,29.753348,-95.379776


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Explore Neighborhoods in Houston

Create a function to repeat the same process to all the neighborhoods in Houston

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called houston_venues

In [21]:
houston_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

FOURTH WARD
SECOND WARD
DOWNTOWN
CLINTON PARK TRI-COMMUNITY
GREATER UPTOWN
GREATER INWOOD
GREATER HOBBY AREA
GOLFCREST / BELLFORT / REVEILLE
ELDRIDGE / WEST OAKS
WASHINGTON AVENUE COALITION / MEMORIAL PARK
GREATER FIFTH WARD
DENVER HARBOR / PORT HOUSTON
PLEASANTVILLE AREA
NORTHSHORE
LAZYBROOK / TIMBERGROVE
GREATER HEIGHTS
KASHMERE GARDENS
MINNETEX
NEAR NORTHSIDE
SPRING BRANCH EAST
SPRING BRANCH NORTH
EL DORADO / OATES PRAIRIE
SPRING BRANCH CENTRAL
HUNTERWOOD
SETTEGAST
LANGWOOD
INDEPENDENCE HEIGHTS
CENTRAL NORTHWEST
TRINITY / HOUSTON GARDENS
CARVERDALE
EASTEX - JENSEN AREA
EAST HOUSTON
ACRES HOME
NORTHSIDE/NORTHLINE
HIDDEN VALLEY
EAST LITTLE YORK / HOMESTEAD
WILLOWBROOK
GREATER GREENSPOINT
IAH / AIRPORT AREA
KINGWOOD AREA
LAKE HOUSTON
FAIRBANKS / NORTHWEST CROSSING
WESTBRANCH
SHARPSTOWN
WESTWOOD
FORT BEND HOUSTON
FONDREN GARDENS
SOUTH BELT / ELLINGTON
SOUTH ACRES / CRESTMONT PARK
BRAYS OAKS
CENTRAL SOUTHWEST
SUNNYSIDE
ALIEF
PECAN PARK
CLEAR LAKE
WESTBURY
WILLOW MEADOWS / WILLOWBEND AREA

Let's check the size of the resulting dataframe

In [22]:
print(houston_venues.shape)
houston_venues.head(10)

(5032, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,FOURTH WARD,29.759108,-95.380204,Eleanor Tinsley Park,29.761440,-95.379271,Park
1,FOURTH WARD,29.759108,-95.380204,Buffalo Bayou Walk,29.762177,-95.375844,Trail
2,FOURTH WARD,29.759108,-95.380204,Buffalo Bayou Cistern,29.762214,-95.375873,Public Art
3,FOURTH WARD,29.759108,-95.380204,Oporto Fooding House & Wine,29.753179,-95.380243,Portuguese Restaurant
4,FOURTH WARD,29.759108,-95.380204,Cafe Poetes,29.753348,-95.379776,Café
5,FOURTH WARD,29.759108,-95.380204,Sam Houston Park,29.759876,-95.371211,Park
6,FOURTH WARD,29.759108,-95.380204,The Fish Restaurant & Sushi Bar,29.752249,-95.376820,Sushi Restaurant
7,FOURTH WARD,29.759108,-95.380204,Paper Street Crossfit,29.757435,-95.385846,Gym
8,FOURTH WARD,29.759108,-95.380204,Lucio's BYOB,29.758326,-95.385591,Bar
9,FOURTH WARD,29.759108,-95.380204,Catalina Coffee,29.767968,-95.380662,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [23]:
houston_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ACRES HOME,15,15,15,15,15,15
ADDICKS PARK TEN,40,40,40,40,40,40
AFTON OAKS / RIVER OAKS AREA,88,88,88,88,88,88
ALIEF,39,39,39,39,39,39
ASTRODOME AREA,100,100,100,100,100,100
BRAEBURN,62,62,62,62,62,62
BRAESWOOD,94,94,94,94,94,94
BRAYS OAKS,43,43,43,43,43,43
BRIAR FOREST,53,53,53,53,53,53


Let's find out how many unique categories can be curated from all the returned venue

In [24]:
print('There are {} uniques categories.'.format(len(houston_venues['Venue Category'].unique())))

There are 318 uniques categories.


Let's put that into a pandas dataframe

In [25]:
# one hot encoding
houston_onehot = pd.get_dummies(houston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
houston_onehot['Neighborhood'] = houston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [houston_onehot.columns[-1]] + list(houston_onehot.columns[:-1])
houston_onehot = houston_onehot[fixed_columns]

houston_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bay,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Football Field,College Rec Center,College Residence Hall,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant,Event Service,Exhibit,Eye Doctor,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,S

Let's examine the new dataframe size.

In [26]:
houston_onehot.shape

(5032, 318)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
houston_grouped = houston_onehot.groupby('Neighborhood').mean().reset_index()
houston_grouped

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bay,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Football Field,College Rec Center,College Residence Hall,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant,Event Service,Exhibit,Eye Doctor,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,S

Let's confirm the new size

In [28]:
houston_grouped.shape

(88, 318)

Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in houston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = houston_grouped[houston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ACRES HOME----
                        venue  freq
0              Discount Store  0.20
1         Fried Chicken Joint  0.13
2  Construction & Landscaping  0.07
3             Thai Restaurant  0.07
4              Baseball Field  0.07


----ADDICKS PARK TEN----
                  venue  freq
0  Fast Food Restaurant  0.08
1    Mexican Restaurant  0.08
2         Grocery Store  0.05
3      Sushi Restaurant  0.05
4                  Park  0.05


----AFTON OAKS / RIVER OAKS AREA----
                     venue  freq
0                     Park  0.05
1                     Bank  0.05
2              Coffee Shop  0.03
3               Donut Shop  0.03
4  New American Restaurant  0.03


----ALIEF----
                  venue  freq
0           Pizza Place  0.08
1        Discount Store  0.08
2  Fast Food Restaurant  0.08
3              Pharmacy  0.05
4            Taco Place  0.05


----ASTRODOME AREA----
                venue  freq
0         Pizza Place  0.05
1        Burger Joint  0.05
2  Mexican Resta

                  venue  freq
0   Fried Chicken Joint  0.18
1            Donut Shop  0.09
2      Business Service  0.09
3        Ice Cream Shop  0.09
4  Fast Food Restaurant  0.09


----IAH / AIRPORT AREA----
                  venue  freq
0           Coffee Shop  0.18
1        Airport Lounge  0.10
2        Sandwich Place  0.06
3  Fast Food Restaurant  0.05
4       Airport Service  0.05


----INDEPENDENCE HEIGHTS----
                  venue  freq
0        Discount Store  0.08
1     Mobile Phone Shop  0.08
2   Fried Chicken Joint  0.08
3  Fast Food Restaurant  0.06
4    Mexican Restaurant  0.06


----KASHMERE GARDENS----
                  venue  freq
0   Fried Chicken Joint  0.13
1        Discount Store  0.13
2  Fast Food Restaurant  0.13
3              Bus Stop  0.09
4                  Food  0.04


----KINGWOOD AREA----
                venue  freq
0         Pizza Place  0.04
1    Asian Restaurant  0.04
2               Hotel  0.04
3  Mexican Restaurant  0.04
4  Italian Restaurant  0.03



                  venue  freq
0   Rental Car Location  0.05
1     Mobile Phone Shop  0.05
2   Fried Chicken Joint  0.05
3              Pharmacy  0.05
4  Fast Food Restaurant  0.04


----WILLOW MEADOWS / WILLOWBEND AREA----
                  venue  freq
0          Burger Joint  0.05
1  Fast Food Restaurant  0.05
2        Clothing Store  0.04
3           Video Store  0.03
4      Department Store  0.03


----WILLOWBROOK----
                venue  freq
0  Mexican Restaurant  0.04
1      Clothing Store  0.04
2        Burger Joint  0.03
3  Chinese Restaurant  0.03
4      Cosmetics Shop  0.03




Let's put that into a pandas dataframe

Let's write a function to sort the venues in descending order

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = houston_grouped['Neighborhood']

for ind in np.arange(houston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(houston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ACRES HOME,Discount Store,Fried Chicken Joint,Park,Liquor Store,Baseball Field,Thai Restaurant,Intersection,Department Store,Pizza Place,Wings Joint
1,ADDICKS PARK TEN,Mexican Restaurant,Fast Food Restaurant,Park,Grocery Store,Sushi Restaurant,Dog Run,Diner,Bakery,Bank,Rental Car Location
2,AFTON OAKS / RIVER OAKS AREA,Bank,Park,New American Restaurant,Trail,Cosmetics Shop,American Restaurant,Coffee Shop,Sandwich Place,Donut Shop,Sushi Restaurant
3,ALIEF,Pizza Place,Fast Food Restaurant,Discount Store,Video Store,Grocery Store,Taco Place,Fried Chicken Joint,Pharmacy,Diner,Chinese Restaurant
4,ASTRODOME AREA,Burger Joint,Pizza Place,Mexican Restaurant,Food Service,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Sandwich Place,Restaurant,BBQ Joint
5,BRAEBURN,Pharmacy,Discount Store,Pizza Place,Liquor Store,Grocery Store,Sandwich Place,Gym / Fitness Center,Chinese Restaurant,Gym,Bank
6,BRAESWOOD,Park,Gym,Pharmacy,Mexican Restaurant,Spa,Ice Cream Shop,Bakery,Pizza Place,Italian Restaurant,Grocery Store
7,BRAYS OAKS,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Gas Station,Pizza Place,Burger Joint,Sandwich Place,Discount Store,Pharmacy,Nightclub
8,BRIAR FOREST,Cosmetics Shop,Sandwich Place,Coffee Shop,Kids Store,Trail,Bank,Clothing Store,Bakery,Grocery Store,Pharmacy
9,CARVERDALE,Home Service,Donut Shop,Gas Station,Burger Joint,Hotel,Business Service,Sandwich Place,Music Store,Restaurant,Intersection


### 3.3	Cluster Neighborhoods Using K-Means Clustering

### Run k-means to cluster the neighborhood into 5 clusters.

In [32]:
# set number of clusters
kclusters = 7

houston_grouped_clustering = houston_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(houston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 4, 3, 6, 4, 4, 4, 6, 3, 4], dtype=int32)

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# add clustering labels
#neighborhoods_venues_sorted.drop('Cluster Labels')
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

houston_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
houston_merged = houston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

houston_merged.head(10) # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,FOURTH WARD,29.759108,-95.380204,3,Coffee Shop,Burger Joint,Pizza Place,Park,Beer Garden,American Restaurant,Brewery,Bar,Sandwich Place,Theater
1,SECOND WARD,29.751715,-95.331634,4,Mexican Restaurant,Music Venue,Gas Station,Park,Fast Food Restaurant,Video Store,Bar,Café,Gym,Discount Store
2,DOWNTOWN,29.758336,-95.353891,3,Hotel,Mexican Restaurant,Bar,Coffee Shop,Brewery,Pizza Place,Baseball Stadium,Beer Garden,Gym,Vietnamese Restaurant
3,CLINTON PARK TRI-COMMUNITY,29.739786,-95.255946,6,Boat or Ferry,Harbor / Marina,Discount Store,Sandwich Place,Baseball Field,Restaurant,Grocery Store,Park,Gas Station,Fast Food Restaurant
4,GREATER UPTOWN,29.771256,-95.490082,4,Pharmacy,Gym / Fitness Center,Coffee Shop,Yoga Studio,Fast Food Restaurant,Mexican Restaurant,Liquor Store,Sandwich Place,Sushi Restaurant,Intersection
5,GREATER INWOOD,29.881288,-95.487484,6,Pharmacy,Discount Store,Grocery Store,Fast Food Restaurant,Video Store,Spanish Restaurant,Convenience Store,Cosmetics Shop,Fried Chicken Joint,Gas Station
6,GREATER HOBBY AREA,29.625936,-95.291085,6,Food Truck,Gas Station,Mexican Restaurant,Airport Terminal,Fast Food Restaurant,Factory,Business Service,Donut Shop,Sandwich Place,Ice Cream Shop
7,GOLFCREST / BELLFORT / REVEILLE,29.653929,-95.298948,0,Rental Car Location,Gas Station,Airport Terminal,Mexican Restaurant,Donut Shop,Airport,History Museum,Park,Discount Store,Burger Joint
8,ELDRIDGE / WEST OAKS,29.737417,-95.670287,3,Bar,Park,Grocery Store,Movie Theater,Athletics & Sports,Italian Restaurant,Gas Station,Vietnamese Restaurant,Dog Run,Coffee Shop
9,WASHINGTON AVENUE COALITION / MEMORIAL PARK,29.765147,-95.418188,3,Bar,Mexican Restaurant,American Restaurant,Gym,Trail,Coffee Shop,New American Restaurant,Art Gallery,Cajun / Creole Restaurant,Golf Course


In [34]:
houston_merged.groupby('Cluster Labels').count()

,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,
0,5,5,5,5,5,5,5,5,5,5,5,5,5
1,10,10,10,10,10,10,10,10,10,10,10,10,10
2,1,1,1,1,1,1,1,1,1,1,1,1,1
3,15,15,15,15,15,15,15,15,15,15,15,15,15
4,35,35,35,35,35,35,35,35,35,35,35,35,35
5,1,1,1,1,1,1,1,1,1,1,1,1,1
6,21,21,21,21,21,21,21,21,21,21,21,21,21


Finally, let's visualize the resulting clusters

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(houston_merged['Latitude'], houston_merged['Longitude'], houston_merged['Neighborhood'], houston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

### Cluster 1

In [36]:
houston_merged.loc[houston_merged['Cluster Labels'] == 0, houston_merged.columns[[0] + list(range(4, houston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,GOLFCREST / BELLFORT / REVEILLE,Rental Car Location,Gas Station,Airport Terminal,Mexican Restaurant,Donut Shop,Airport,History Museum,Park,Discount Store,Burger Joint
21,EL DORADO / OATES PRAIRIE,Rental Car Location,Ice Cream Shop,Gas Station,Sandwich Place,Intersection,Shipping Store,Eye Doctor,Dry Cleaner,Farmers Market,Dive Bar
23,HUNTERWOOD,Fried Chicken Joint,Fast Food Restaurant,Ice Cream Shop,Business Service,Donut Shop,Sandwich Place,Intersection,Rental Car Location,Gas Station,Gym / Fitness Center
31,EAST HOUSTON,American Restaurant,Golf Course,Gas Station,Tree,Rental Car Location,Empanada Restaurant,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant
48,SOUTH ACRES / CRESTMONT PARK,Grocery Store,Discount Store,Health Food Store,Nightclub,Pharmacy,Pet Store,Cocktail Bar,Gas Station,Electronics Store,Dog Run


### Cluster 2

In [37]:
houston_merged.loc[houston_merged['Cluster Labels'] == 1, houston_merged.columns[[0] + list(range(4, houston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,DENVER HARBOR / PORT HOUSTON,Mexican Restaurant,Boat or Ferry,Fast Food Restaurant,Intersection,Convenience Store,Restaurant,Optical Shop,Bay,Baseball Field,Pier
12,PLEASANTVILLE AREA,Rental Service,Gas Station,Mexican Restaurant,Brewery,Taco Place,Fast Food Restaurant,Office,Sandwich Place,Intersection,Fried Chicken Joint
13,NORTHSHORE,Mexican Restaurant,Fast Food Restaurant,Seafood Restaurant,Fried Chicken Joint,Convenience Store,Chinese Restaurant,Discount Store,Gas Station,BBQ Joint,Pizza Place
18,NEAR NORTHSIDE,Mexican Restaurant,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Bank,Food Truck,Bar,Pizza Place,Gas Station,Food
35,EAST LITTLE YORK / HOMESTEAD,Mexican Restaurant,Fast Food Restaurant,Convenience Store,Gas Station,Park,Taco Place,Bakery,Cajun / Creole Restaurant,Discount Store,Wings Joint
62,PARK PLACE,Mexican Restaurant,Gas Station,Discount Store,Ice Cream Shop,Vietnamese Restaurant,Golf Course,Fast Food Restaurant,Convenience Store,Smoke Shop,Electronics Store
67,GULFGATE RIVERVIEW / PINE VALLEY,Fast Food Restaurant,Mexican Restaurant,Convenience Store,Discount Store,Taco Place,Grocery Store,Furniture / Home Store,Light Rail Station,Park,Diner
72,LAWNDALE / WAYSIDE,Mexican Restaurant,Pizza Place,Fried Chicken Joint,Convenience Store,Fast Food Restaurant,Pharmacy,Discount Store,Sandwich Place,Snack Place,Seafood Restaurant
80,EDGEBROOK AREA,Mexican Restaurant,Fast Food Restaurant,Discount Store,Sandwich Place,Bank,Video Store,Rental Car Location,Seafood Restaurant,Burger Joint,Latin American Restaurant
81,MAGNOLIA PARK,Mexican Restaurant,Fried Chicken Joint,Pizza Place,Discount Store,Fast Food Restaurant,Convenience Store,Chinese Restaurant,Sandwich Place,Grocery Store,Cosmetics Shop


### Cluster 3

In [38]:
houston_merged.loc[houston_merged['Cluster Labels'] == 2, houston_merged.columns[[0] + list(range(4, houston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,FORT BEND HOUSTON,Construction & Landscaping,Discount Store,Convenience Store,Fast Food Restaurant,Exhibit,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant


### Cluster 4

In [39]:
houston_merged.loc[houston_merged['Cluster Labels'] == 3, houston_merged.columns[[0] + list(range(4, houston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,FOURTH WARD,Coffee Shop,Burger Joint,Pizza Place,Park,Beer Garden,American Restaurant,Brewery,Bar,Sandwich Place,Theater
2,DOWNTOWN,Hotel,Mexican Restaurant,Bar,Coffee Shop,Brewery,Pizza Place,Baseball Stadium,Beer Garden,Gym,Vietnamese Restaurant
8,ELDRIDGE / WEST OAKS,Bar,Park,Grocery Store,Movie Theater,Athletics & Sports,Italian Restaurant,Gas Station,Vietnamese Restaurant,Dog Run,Coffee Shop
9,WASHINGTON AVENUE COALITION / MEMORIAL PARK,Bar,Mexican Restaurant,American Restaurant,Gym,Trail,Coffee Shop,New American Restaurant,Art Gallery,Cajun / Creole Restaurant,Golf Course
15,GREATER HEIGHTS,Mexican Restaurant,Coffee Shop,Ice Cream Shop,Burger Joint,Cajun / Creole Restaurant,New American Restaurant,Gym / Fitness Center,Bar,American Restaurant,Gift Shop
38,IAH / AIRPORT AREA,Coffee Shop,Airport Lounge,Sandwich Place,Fast Food Restaurant,Airport Terminal,Airport Service,Pizza Place,Restaurant,American Restaurant,Airport
64,MEDICAL CENTER AREA,Zoo Exhibit,Food Truck,Burger Joint,Food Service,Restaurant,Coffee Shop,American Restaurant,Art Gallery,Hotel,Trail
69,UNIVERSITY PLACE,Zoo Exhibit,Coffee Shop,Burger Joint,Bar,Italian Restaurant,Bookstore,Sandwich Place,Food Service,American Restaurant,Wine Bar
71,MUSEUM PARK,Zoo Exhibit,Coffee Shop,Bar,Science Museum,Trail,American Restaurant,Art Gallery,Sculpture Garden,Park,Southern / Soul Food Restaurant
73,GREENWAY / UPPER KIRBY AREA,Coffee Shop,Mexican Restaurant,Café,American Restaurant,Bakery,Gym,Burger Joint,Chinese Restaurant,New American Restaurant,Clothing Store


### Cluster 5

In [40]:
houston_merged.loc[houston_merged['Cluster Labels'] == 4, houston_merged.columns[[0] + list(range(4, houston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,SECOND WARD,Mexican Restaurant,Music Venue,Gas Station,Park,Fast Food Restaurant,Video Store,Bar,Café,Gym,Discount Store
4,GREATER UPTOWN,Pharmacy,Gym / Fitness Center,Coffee Shop,Yoga Studio,Fast Food Restaurant,Mexican Restaurant,Liquor Store,Sandwich Place,Sushi Restaurant,Intersection
14,LAZYBROOK / TIMBERGROVE,Bar,Park,Mexican Restaurant,American Restaurant,Gym / Fitness Center,Fast Food Restaurant,Smoothie Shop,Sandwich Place,Beer Bar,Sports Bar
19,SPRING BRANCH EAST,Mexican Restaurant,Sandwich Place,Pharmacy,Coffee Shop,Italian Restaurant,Pizza Place,Video Store,Donut Shop,Sporting Goods Shop,Sushi Restaurant
20,SPRING BRANCH NORTH,Mexican Restaurant,Korean Restaurant,Pizza Place,Pharmacy,Discount Store,Fried Chicken Joint,Vietnamese Restaurant,Sandwich Place,Grocery Store,Food Truck
22,SPRING BRANCH CENTRAL,Korean Restaurant,Mexican Restaurant,Gym / Fitness Center,Sandwich Place,BBQ Joint,Pharmacy,Bubble Tea Shop,Sporting Goods Shop,Bank,Bakery
25,LANGWOOD,Hotel,Fast Food Restaurant,Sandwich Place,Bank,Donut Shop,Pizza Place,Gas Station,Mexican Restaurant,Sports Bar,Convenience Store
27,CENTRAL NORTHWEST,Hotel,Fast Food Restaurant,Chinese Restaurant,Park,Dive Bar,Gym / Fitness Center,Donut Shop,Burger Joint,Sandwich Place,Pizza Place
29,CARVERDALE,Home Service,Donut Shop,Gas Station,Burger Joint,Hotel,Business Service,Sandwich Place,Music Store,Restaurant,Intersection
36,WILLOWBROOK,Clothing Store,Mexican Restaurant,Burger Joint,Cosmetics Shop,Furniture / Home Store,Coffee Shop,Chinese Restaurant,Latin American Restaurant,Electronics Store,Lingerie Store


### Cluster 6

In [41]:
houston_merged.loc[houston_merged['Cluster Labels'] == 5, houston_merged.columns[[0] + list(range(4, houston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,HARRISBURG / MANCHESTER,Park,Mexican Restaurant,Harbor / Marina,Donut Shop,Restaurant,Athletics & Sports,Pier,Cosmetics Shop,Event Service,Construction & Landscaping


### Cluster 7

In [42]:
houston_merged.loc[houston_merged['Cluster Labels'] == 6, houston_merged.columns[[0] + list(range(4, houston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,CLINTON PARK TRI-COMMUNITY,Boat or Ferry,Harbor / Marina,Discount Store,Sandwich Place,Baseball Field,Restaurant,Grocery Store,Park,Gas Station,Fast Food Restaurant
5,GREATER INWOOD,Pharmacy,Discount Store,Grocery Store,Fast Food Restaurant,Video Store,Spanish Restaurant,Convenience Store,Cosmetics Shop,Fried Chicken Joint,Gas Station
6,GREATER HOBBY AREA,Food Truck,Gas Station,Mexican Restaurant,Airport Terminal,Fast Food Restaurant,Factory,Business Service,Donut Shop,Sandwich Place,Ice Cream Shop
10,GREATER FIFTH WARD,Park,Sandwich Place,Pharmacy,Fast Food Restaurant,Gas Station,Intersection,Video Store,Discount Store,Music Venue,Beer Garden
16,KASHMERE GARDENS,Fast Food Restaurant,Discount Store,Fried Chicken Joint,Bus Stop,Grocery Store,Gas Station,Nightclub,Chinese Restaurant,Intersection,Restaurant
17,MINNETEX,Fast Food Restaurant,Golf Driving Range,Shipping Store,Locksmith,Liquor Store,Fried Chicken Joint,Donut Shop,Baseball Field,Lake,Discount Store
24,SETTEGAST,Convenience Store,Burger Joint,American Restaurant,BBQ Joint,Motel,Gas Station,Discount Store,Sandwich Place,Fried Chicken Joint,Event Service
26,INDEPENDENCE HEIGHTS,Discount Store,Fried Chicken Joint,Mobile Phone Shop,Fast Food Restaurant,Mexican Restaurant,Sandwich Place,BBQ Joint,Pharmacy,American Restaurant,Breakfast Spot
28,TRINITY / HOUSTON GARDENS,American Restaurant,Discount Store,Burger Joint,Fried Chicken Joint,BBQ Joint,Gas Station,Sandwich Place,Home Service,Motel,Convenience Store
30,EASTEX - JENSEN AREA,Fast Food Restaurant,Pizza Place,Discount Store,Pharmacy,Mexican Restaurant,Intersection,Brewery,Construction & Landscaping,Flea Market,Supermarket


## 5	Discussion
In this project, we have directed our efforts on detecting the most common venue category of each neighborhood of Houston. We have collected the required data and explored the neighborhoods and segment them using Foursquare API. We have explored the venues across different areas of Houston. After creating the dataframe to display the top 10 venues for each neighborhood, our analysis shows although there is a great number of different types of venue category in neighborhoods of Houston, these neighborhoods can be clustered to 7 different groups based on their most common venue categories.
Based on our clustering results: Cluster 1, where the rental car store and gas station are the most common venue categories, is the “suburb” area of Houston. Cluster 2 and Cluster 7, where fast food restaurant are the most common venue categories, are the “semi-suburb” area of Houston. Cluster 3 and cluster 6, which only contains one neighborhood each, can be considered as “outliners” in the analysis. Cluster 4, which features hotel, bar and coffee shop, is the “modern” area of Houston. Cluster 5, which features different varieties of restaurants, is the “restaurant” area of Houston.
Therefore, based on previous analysis result, tourists can select their choice of location to visit in Houston. People who like to taste varieties of food can choose cluster 5. People who enjoy the modern city feel, can visit cluster 4. 

## 6	Conclusion
The purpose of this project was to identify the most common venue categories and clustering neighborhoods of Houston, in order to aid stakeholders in narrowing down the place to visit in the neighborhoods of Houston. To help arrive at this solution, K-means clustering was used to group neighborhoods in Houston. The visualization library was also used to visualize the clustering results.
